In [77]:
import base64
import re
import math  

# Here is how Wiener RSA Attack works (ref: https://en.wikipedia.org/wiki/Wiener%27s_attack)
# Using Wiener's Theorem and continued fractions to approximate d, first we find the continued fractions expansion of e / N.
# Then, we find all convergents k / D. (formula for convergents on: https://en.wikipedia.org/wiki/Continued_fraction)

N = 194749497518847283
e = 50736902528669041
C = 'Qz1bNDc0MDYyNjMxOTI2OTM1MDksNTEwNjUxNzgyMDExNzIyMjMsMzAyNjA1NjUyMzUxMjg3MDQsODIzODU5NjMzMzQ0MDQyNjgNCjgxNjkxNTY2NjM5Mjc5MjksNDc0MDYyNjMxOTI2OTM1MDksMTc4Mjc1OTc3MzM2Njk2NDQyLDEzNDQzNDI5NTg5NDgwMzgwNg0KMTEyMTExNTcxODM1NTEyMzA3LDExOTM5MTE1MTc2MTA1MDg4MiwzMDI2MDU2NTIzNTEyODcwNCw4MjM4NTk2MzMzNDQwNDI2OA0KMTM0NDM0Mjk1ODk0ODAzODA2LDQ3NDA2MjYzMTkyNjkzNTA5LDQ1ODE1MzIwOTcyNTYwMjAyLDE3NDYzMjIyOTMxMjA0MTI0OA0KMzAyNjA1NjUyMzUxMjg3MDQsNDc0MDYyNjMxOTI2OTM1MDksMTE5MzkxMTUxNzYxMDUwODgyLDU3MjA4MDc3NzY2NTg1MzA2DQoxMzQ0MzQyOTU4OTQ4MDM4MDYsNDc0MDYyNjMxOTI2OTM1MDksMTE5MzkxMTUxNzYxMDUwODgyLDQ3NDA2MjYzMTkyNjkzNTA5DQoxMTIxMTE1NzE4MzU1MTIzMDcsNTI4ODI4NTEwMjYwNzI1MDcsMTE5MzkxMTUxNzYxMDUwODgyLDU3MjA4MDc3NzY2NTg1MzA2DQoxMTkzOTExNTE3NjEwNTA4ODIsMTEyMTExNTcxODM1NTEyMzA3LDgxNjkxNTY2NjM5Mjc5MjksMTM0NDM0Mjk1ODk0ODAzODA2DQo1NzIwODA3Nzc2NjU4NTMwNiw0NzQwNjI2MzE5MjY5MzUwOSwxODU1ODIxMDUyNzUwNTA5MzIsMTc0NjMyMjI5MzEyMDQxMjQ4DQoxMzQ0MzQyOTU4OTQ4MDM4MDYsODIzODU5NjMzMzQ0MDQyNjgsMTcyNTY1Mzg2MzkzNDQzNjI0LDEwNjM1NjUwMTg5MzU0NjQwMQ0KODE2OTE1NjY2MzkyNzkyOSw0NzQwNjI2MzE5MjY5MzUwOSwxMDM2MTA1OTcyMDYxMDgxNiwxMzQ0MzQyOTU4OTQ4MDM4MDYNCjExOTM5MTE1MTc2MTA1MDg4MiwxNzI1NjUzODYzOTM0NDM2MjQsNDc0MDYyNjMxOTI2OTM1MDksODE2OTE1NjY2MzkyNzkyOQ0KNTI4ODI4NTEwMjYwNzI1MDcsMTE5MzkxMTUxNzYxMDUwODgyLDgxNjkxNTY2NjM5Mjc5MjksNDc0MDYyNjMxOTI2OTM1MDkNCjQ1ODE1MzIwOTcyNTYwMjAyLDE3NDYzMjIyOTMxMjA0MTI0OCwzMDI2MDU2NTIzNTEyODcwNCw0NzQwNjI2MzE5MjY5MzUwOQ0KNTI4ODI4NTEwMjYwNzI1MDcsMTE5MzkxMTUxNzYxMDUwODgyLDExMTUyMzQwODIxMjQ4MTg3OSwxMzQ0MzQyOTU4OTQ4MDM4MDYNCjQ3NDA2MjYzMTkyNjkzNTA5LDExMjExMTU3MTgzNTUxMjMwNyw1Mjg4Mjg1MTAyNjA3MjUwNywxMTkzOTExNTE3NjEwNTA4ODINCjU3MjA4MDc3NzY2NTg1MzA2LDExOTM5MTE1MTc2MTA1MDg4MiwxMTIxMTE1NzE4MzU1MTIzMDcsODE2OTE1NjY2MzkyNzkyOQ0KMTM0NDM0Mjk1ODk0ODAzODA2LDU3MjA4MDc3NzY2NTg1MzA2XQ=='

# The fast() function from exercise 8(iv) to compute remainder of ( a^g modulo N )

def fast(a,g,N):
    g = bin(g)         # Turn to binary
    g = g[2:]          # Remove the 0b at the beginning

    d = 1
    x = a
    for i in range (len(g)-1,-1,-1):
        if g[i]==str(1):
            d = (d * x) % N
        x = (x ** 2) % N
    return d


# Here i calculate the continued fraction of e/N

def continued_fraction(e, N):
    
    integers=[]
    while N>0:
        a = e // N
        integers.append(a)
        temp = e
        e = N
        N = temp - a * N
    return integers


# Here I compute the convergents based on the continued fraction list given.

def convergents(frac_list):

    convs = [];
    for i in range (0,len(frac_list)):
        convs.append(formula(frac_list[0:i+1]))
    return convs


# num = numerator, denom = denominator

def formula (frac_list):
    
    num = frac_list[-1]
    den = 1
    for i in range(-2,-len(frac_list)-1,-1):
        num, den = frac_list[i]*num+den, num
    return (num,den)


# Here i build the Wiener's attack algorithm.
# To check if d is actually the key, I need to compute x^2 - (N - phi + 1)x + N = 0
# If it has 2 integer roots, then we have successfully found d.

def find_d (convs,e,n):
    
    for (k,d) in convs:
        if k!=0 and (e*d-1)%k == 0:         # If phi ( (e*d - 1) / N ) is an integer
            phi = (e*d-1)//k
            b = n - phi + 1
            delta = b*b - 4*1*n             # discriminant = b^2 - 4*a*g
            if(delta>=0):                   # If discriminant is an integer
                t = math.sqrt(delta)
                if t*t == delta and (b+t)%2==0:
                    return d                # We successfully found d

                
# Find continued fraction of e/N, find its convergents, and finally find the d
                
confrac = continued_fraction(e,N)
convs = convergents(confrac)
d = find_d(convs,e,N)

# Decode with base64 (ref: https://stackabuse.com/encoding-and-decoding-base64-strings-in-python)

base64_bytes = C.encode('ascii')
message_bytes = base64.b64decode(base64_bytes)
message = message_bytes.decode('ascii')
message = message[3:-1]                  # Remove the C=[] from the string

encrypted = re.split('\r\n|,',message)   # Pipeline to clean our string and turn into a list

# Show hidden message

hiddenMessage=''
for i in encrypted:
    k = fast(int(i),d,N)
    hiddenMessage = hiddenMessage + chr(k)
    
print("Hidden Message:\n\n" + hiddenMessage)

Hidden Message:

 Just because you are a character doesn't mean that you have character
